In [2]:
import requests
import json
import prettytable
from bs4 import BeautifulSoup as bs
import pymongo
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pygeoj
import re

In [3]:
filepath = "mw_scraped_indeedjobs.csv"
# filepath = "https://mw-project-4.s3.us-west-1.amazonaws.com/mw_job_listings_scraped.csv"
# changing filepath to save load time
df = pd.read_csv(filepath)
df = df.sort_values('Job Category')

df["Job Category"]= df["Job Category"].str.lower() 
df["Job Title"]= df["Job Title"].str.lower() 
df['Job Category'].value_counts()

database admin                     1536
big data software engineer         1201
quantitative analyst               1150
it systems analyst                 1082
data analyst                       1076
business intelligence analyst      1074
data architect                     1068
business systems analyst           1047
machine learning engineer          1043
data engineer                      1040
data scientist                     1031
machine learning scientist         1031
business intelligence developer    1026
logistics analyst                  1024
marketing analyst                  1023
data warehouse manager             1021
statistician                       1017
data storyteller                   1016
database administrator              859
database manager                    442
data modeler                        438
Name: Job Category, dtype: int64

In [4]:
salarydf = df.dropna()
salarydf

,Job Category,Job Title,Company Name,Company Location,Salary
4347,big data software engineer,senior software engineer - big data - ranking ...,Indeed,"Austin, TX","$132,000 - $192,000 a year"
5019,big data software engineer,"software engineer, deep learning infrastructur...",Tesla,"Palo Alto, CA",Estimated $121K â€“ $154K a year
5017,big data software engineer,software engineer,Microsoft,"Honolulu, HI",Estimated $77.8K â€“ $98.4K a year
5016,big data software engineer,"lead software engineer, casino (web)",DraftKings,"Boston, MA",Estimated $133K â€“ $168K a year
5020,big data software engineer,web/full stack engineer,Tinder,"West Hollywood, CA",Estimated $124K â€“ $157K a year
...,...,...,...,...,...
15206,statistician,statistician ii,USC,"Los Angeles, CA",Estimated $90.9K â€“ $115K a year
15207,statistician,research scientist - rare neurogenetic disease,RTI International,"Durham, NC",Estimated $83.5K â€“ $106K a year
15208,statistician,"statistician, quality analytics",Invitae,"Boulder, CO","From $55,000 a year"
15209,statistician,sas programmer,NoyMed CRO,Remote,"$36,000 - $55,000 a year"


In [10]:
test = salarydf[salarydf['Company Location'] == 'Fort Meade, MD']
test1 = salarydf[salarydf['Company Name'] == 'ArthurPaige']
test1

,Job Category,Job Title,Company Name,Company Location,Salary
2405,data scientist,senior data scientist,ArthurPaige,"Fort Meade, MD","$190,000 - $240,000 a year"
2396,data scientist,senior data scientist,ArthurPaige,"Fort Meade, MD","$190,000 - $240,000 a year"


In [21]:
dfa = salarydf[salarydf['Salary'].str.contains("a year")]
dfb = salarydf[salarydf['Salary'].str.contains("an hour")]
# test= salarydf['Salary'].apply(lambda x: x.replace('K','').replace('$','').replace(",","").replace("â€“","-"))
year = dfa['Salary'].apply(lambda x: x.replace('K','').replace(",","").replace("â€“","-").replace("000", ""))
salarydf['min_salary'] = year.apply(lambda x: float(x.split('-')[0].split('$')[1].split(' a')[0].split(',')[0]))
# salarydf[salarydf['min_salary'].str.contains("a year")]
test = year[year.str.contains('-')]

salarydf['max_salary'] = test.apply(lambda x: float(x.split('-')[1].split('$')[1].split(' a')[0].split(',')[0]))

# salarydf
yeardf = salarydf.dropna()
yeardf = yeardf.drop(yeardf[yeardf.min_salary > 999].index)
yeardf = yeardf.drop(yeardf[yeardf.max_salary > 999].index)

yeardf['Mean Salary'] = yeardf[['min_salary', 'max_salary']].mean(axis=1)
yeardf

yeardf.to_csv('new.csv', index = False)

In [22]:
# yeardf['Mean Salary'].max()
yeardf

,Job Category,Job Title,Company Name,Area text,Salary,min_salary,max_salary,Mean Salary
4347,Big Data Software Engineer,Senior Software Engineer - Big Data - Ranking ...,Indeed,"Austin, TX","$132,000 - $192,000 a year",132.0,192.0,162.00
5019,Big Data Software Engineer,"Software Engineer, Deep Learning Infrastructur...",Tesla,"Palo Alto, CA",Estimated $121K â€“ $154K a year,121.0,154.0,137.50
5017,Big Data Software Engineer,Software Engineer,Microsoft,"Honolulu, HI",Estimated $77.8K â€“ $98.4K a year,77.8,98.4,88.10
5016,Big Data Software Engineer,"LEAD SOFTWARE ENGINEER, CASINO (WEB)",DraftKings,"Boston, MA",Estimated $133K â€“ $168K a year,133.0,168.0,150.50
5020,Big Data Software Engineer,Web/Full Stack Engineer,Tinder,"West Hollywood, CA",Estimated $124K â€“ $157K a year,124.0,157.0,140.50
...,...,...,...,...,...,...,...,...
15212,Statistician,Biostatistician (Research Associate),Oregon Health & Science University,"Portland, OR",Estimated $56.5K â€“ $71.5K a year,56.5,71.5,64.00
15206,Statistician,Statistician II,USC,"Los Angeles, CA",Estimated $90.9K â€“ $115K a year,90.9,115.0,102.95
15207,Statistician,Research Scientist - Rare Neurogenetic Disease,RTI International,"Durham, NC",Estimated $83.5K â€“ $106K a year,83.5,106.0,94.75
15209,Statistician,SAS Programmer,NoyMed CRO,Remote,"$36,000 - $55,000 a year",36.0,55.0,45.50


In [23]:
df = yeardf
df1=df[df['Area text'].str.contains(", CA")]
dfca=df1
dfca['Area text']="CA"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", AL")]
dfal=df1
dfal['Area text']="AL"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", AK")]
df2=df[df['Area text'].str.contains("Alaska")]
dfak=df1
dfak['Area text']="AK"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", AZ")]
df2=df[df['Area text'].str.contains("Arizona")]
dfaz=df1
dfaz['Area text']="AZ"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", AR")]
df2=df[df['Area text'].str.contains("Arkansas")]
dfar=df1
dfar['Area text']="AR"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", CO")]
df2=df[df['Area text'].str.contains("Colorado")]
dfco=df1
dfco['Area text']="CO"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", CT")]
df2=df[df['Area text'].str.contains("Connecticut")]
dfct=df1
dfct['Area text']="CT"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", DE")]
df2=df[df['Area text'].str.contains("Delaware")]
dfde=df1
dfde['Area text']="DE"
# dfca['Occupation text'].value_counts().reset_index()

# dfdc=df[df['Area text'].str.contains("District of Columbia")]
# dfdc['Area text']="DC"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", FL")]
df2=df[df['Area text'].str.contains("Florida")]
dffl=df1
dffl['Area text']="FL"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", GA")]
df2=df[df['Area text'].str.contains("Georgia")]
dfga=df1
dfga['Area text']="GA"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", HI")]
df2=df[df['Area text'].str.contains("Hawaii")]
dfhi=df1
dfhi['Area text']="HI"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", ID")]
df2=df[df['Area text'].str.contains("Idaho")]
dfid=df1
dfid['Area text']="ID"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", IL")]
df2=df[df['Area text'].str.contains("Illinois")]
dfil=df1
dfil['Area text']="IL"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", IN")]
df2=df[df['Area text'].str.contains("Indiana")]
df3=df2[df2['Area text']!='Indianapolis-Carmel-Anderson, IN']
dfin=df1
dfin['Area text']="IN"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", IA")]
df2=df[df['Area text'].str.contains("Iowa")]
df3=df2[df2['Area text']!="Iowa City, IA"]
dfia=df1
dfia['Area text']="IA"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", KS")]
df2=df[df['Area text'].str.contains("Kansas")]
dfks=df1
dfks['Area text']="KS"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", KY")]
df2=df[df['Area text'].str.contains("Kentucky")]
dfky=df1
dfky['Area text']="KY"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", LA")]
df2=df[df['Area text'].str.contains("Louisiana")]
dfla=df1
dfla['Area text']="LA"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", ME")]
df2=df[df['Area text'].str.contains("Maine")]
dfme=df1
dfme['Area text']="ME"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", MD")]
df2=df[df['Area text'].str.contains("Maryland")]
dfmd=df1
dfmd['Area text']="MD"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", MA")]
df2=df[df['Area text'].str.contains("Massachusetts")]
dfma=df1
dfma['Area text']="MA"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", MI")]
df2=df[df['Area text'].str.contains("Michigan")]
df3=df2[df2['Area text']!='Michigan City-La Porte, IN']
dfmi=df1
dfmi['Area text']="MI"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", MN")]
df2=df[df['Area text'].str.contains("Minnesota")]
dfmn=df1
dfmn['Area text']="MN"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", MS")]
df2=df[df['Area text'].str.contains("Mississippi")]
dfms=df1
dfms['Area text']="MS"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", MO")]
df2=df[df['Area text'].str.contains("Missouri")]
dfmo=df1
dfmo['Area text']="MO"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", MT")]
df2=df[df['Area text'].str.contains("Montana")]
dfmt=df1
dfmt['Area text']="MT"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", NE")]
df2=df[df['Area text'].str.contains("Nebraska")]
dfne=df1
dfne['Area text']="NE"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", NV")]
df2=df[df['Area text'].str.contains("Nevada")]
dfnv=df1
dfnv['Area text']="NV"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", NH")]
df2=df[df['Area text'].str.contains("New Hampshire")]
dfnh=df1
dfnh['Area text']="NH"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", NJ")]
df2=df[df['Area text'].str.contains("New Jersey")]
dfnj=df1
dfnj['Area text']="NJ"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", NM")]
df2=df[df['Area text'].str.contains("New Mexico")]
dfnm=df1
dfnm['Area text']="NM"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", NY")]
df2=df[df['Area text'].str.contains("New York")]
df3=df2[df2['Area text']!='New York-Newark-Jersey City, NY-NJ-PA']
dfny=df1
dfny['Area text']="NY"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", NC")]
df2=df[df['Area text'].str.contains("North Carolina")]
dfnc=df1
dfnc['Area text']="NC"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", ND")]
df2=df[df['Area text'].str.contains("North Dakota")]
dfnd=df1
dfnd['Area text']="ND"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", OH")]
df2=df[df['Area text'].str.contains("Ohio")]
dfoh=df1
dfoh['Area text']="OH"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", OK")]
df2=df[df['Area text'].str.contains("Oklahoma")]
df3=df2[df2['Area text']!='Oklahoma City, OK']
dfok=df1
dfok['Area text']="OK"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", OR")]
df2=df[df['Area text'].str.contains("Oregon")]
dfor=df1
dfor['Area text']="OR"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", PA")]
df2=df[df['Area text'].str.contains("Pennsylvania")]
dfpa=df1
dfpa['Area text']="PA"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", RI")]
df2=df[df['Area text'].str.contains("Rhode Island")]
dfri=df1
dfri['Area text']="RI"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", SC")]
df2=df[df['Area text'].str.contains("South Carolina")]
dfsc=df1
dfsc['Area text']="SC"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", SD")]
df2=df[df['Area text'].str.contains("South Dakota")]
dfsd=df1
dfsd['Area text']="SD"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", TN")]
df2=df[df['Area text'].str.contains("Tennessee")]
dftn=df1
dftn['Area text']="TN"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", TX")]
df2=df[df['Area text'].str.contains("Texas")]
dftx=df1
dftx['Area text']="TX"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", UT")]
df2=df[df['Area text'].str.contains("Utah")]
dfut=df1
dfut['Area text']="UT"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", VT")]
df2=df[df['Area text'].str.contains("Vermont")]
dfvt=df1
dfvt['Area text']="VT"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", VA")]
df2=df[df['Area text'].str.contains("Virginia")]
df3=df2[df2['Area text']!='Virginia Beach-Norfolk-Newport News, VA-NC']
df4=df3[df3['Area text']!='Northern West Virginia nonmetropolitan area']
df5=df4[df4['Area text']!='Southern West Virginia nonmetropolitan area']
df6=df5[df5['Area text']!='West Virginia']
dfva=df1
dfva['Area text']="VA"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", WA")]
df2=df[df['Area text'].str.contains("Washington")]
dfwa=df1
dfwa['Area text']="WA"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", WV")]
df2=df[df['Area text'].str.contains("West Virginia")]
dfwv=df1
dfwv['Area text']="WV"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", WI")]
df2=df[df['Area text'].str.contains("Wisconsin")]
dfwi=df1
dfwi['Area text']="WI"
# dfca['Occupation text'].value_counts().reset_index()

df1=df[df['Area text'].str.contains(", WY")]
df2=df[df['Area text'].str.contains("Wyoming")]
df3=df2[df2['Area text']!='Grand Rapids-Wyoming, MI']
dfwy=df1
dfwy['Area text']="WY"
# dfca['Occupation text'].value_counts().reset_index()

dfremote = df[df['Area text'].str.contains("Remote")]

<ipython-input-23-5a2ffe201d99>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfca['Area text']="CA"
<ipython-input-23-5a2ffe201d99>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfal['Area text']="AL"
<ipython-input-23-5a2ffe201d99>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [24]:
states = [dfal, dfak, dfaz, dfar, dfca, dfco, dfct, dfde, dffl, dfga, dfhi, dfid, dfil, dfin, dfia, dfks, dfky, dfla, dfme, dfmd, dfma, dfmi, dfmn, dfms, dfmo, dfmt, dfne, dfnv, dfnh, dfnj, dfnm, dfny, dfnc, dfnd, dfoh, dfok, dfor, dfpa, dfri, dfsc, dfsd, dftn, dftx, dfut, dfvt, dfva, dfwa, dfwv, dfwi, dfwy, dfremote] 
statesdf = pd.concat(states)
statesdf = statesdf.rename(columns = {'Area text': 'State'} )
statesdf.to_csv('statesdata.csv', index = False)

In [25]:
statesdf

,Job Category,Job Title,Company Name,State,Salary,min_salary,max_salary,Mean Salary
21000,Business Intelligence Developer,Business Intelligence Developer,Newport Group,AL,Estimated $79.7K â€“ $101K a year,79.7,101.0,90.35
3316,Data Architect,Database Developer,CyberCoders,AL,"$80,000 - $100,000 a year",80.0,100.0,90.00
2209,Data Scientist,Data Scientist,"Davidson Technologies, Inc.",AL,"$100,000 - $200,000 a year",100.0,200.0,150.00
13203,Database Administrator,Database Administrator,"Geographic Information Services, Inc.",AL,Estimated $81.8K â€“ $104K a year,81.8,104.0,92.90
8091,Logistics Analyst,"Logistics Analyst (Basic), Redstone Arsenal, AL","CALIBRE Systems, Inc.",AL,Estimated $47.5K â€“ $60.1K a year,47.5,60.1,53.80
...,...,...,...,...,...,...,...,...
15227,Statistician,Biostatistician/Biostatistician II/Senior Bios...,IQVIA,Remote,Estimated $130K â€“ $165K a year,130.0,165.0,147.50
15225,Statistician,Data Analyst (Data Science),Natera,Remote,Estimated $67.6K â€“ $85.6K a year,67.6,85.6,76.60
15224,Statistician,"Senior Software Engineer, Ads Optimization",Reddit,Remote,Estimated $124K â€“ $157K a year,124.0,157.0,140.50
15222,Statistician,"Sr. Manager, Publication and Research",Emergency Care Research Institute,Remote,Estimated $76.8K â€“ $97.2K a year,76.8,97.2,87.00


In [ ]:
states = [dfal, dfak, dfaz, dfar, dfca, dfco, dfct, dfde, dffl, dfga, dfhi, dfid, dfil, dfin, dfia, dfks, dfky, dfla, dfme, dfmd, dfma, dfmi, dfmn, dfms, dfmo, dfmt, dfne, dfnv, dfnh, dfnj, dfnm, dfny, dfnc, dfnd, dfoh, dfok, dfor, dfpa, dfri, dfsc, dfsd, dftn, dftx, dfut, dfvt, dfva, dfwa, dfwv, dfwi, dfwy] 
for z in states: 
    z['Code'] = z['Area text'] + z['Job Category']
test = dfal['Code'].value_counts()

In [ ]:
states = [dfal, dfak, dfaz, dfar, dfca, dfco, dfct, dfde, dffl, dfga, dfhi, dfid, dfil, dfin, dfia, dfks, dfky, dfla, dfme, dfmd, dfma, dfmi, dfmn, dfms, dfmo, dfmt, dfne, dfnv, dfnh, dfnj, dfnm, dfny, dfnc, dfnd, dfoh, dfok, dfor, dfpa, dfri, dfsc, dfsd, dftn, dftx, dfut, dfvt, dfva, dfwa, dfwv, dfwi, dfwy]    
# geojsonpath = '../resources/geojsons/states.geojson'
# geojson1 = pd.read_json(geojsonpath)
newtest = pygeoj.load('../resources/geojsons/test.geojson')

# for x in states:
#     print(x)
#     teststate = x.groupby(['Occupation text'])  
#     teststate2 = teststate["Job characteristic code"].count().reset_index().sort_values('Occupation text').set_index('Occupation text')
#     testdict = teststate2.to_dict()
#     index = states.index(x)+1
#     print(index)
#     newtest[index].properties = testdict['Job characteristic code']
#     newtest[index].properties['jobs'] = testdict

def Merge(a, b):
    return(b.update(a))
counter=0
added=[]

for x in states:
    teststate = x.groupby(['Job Category'])
    teststate2 = teststate["Code"].count().reset_index().sort_values('Job Category').set_index('Job Category')
    testdict = teststate2.to_dict()
    added.append(testdict)
# #     print(testdict)
#     newtest[y].properties=testdict['Job characteristic code']
#     newtest[y].properties['jobs']=testdict
#     y=+1

for y in newtest:
        
        Merge(added[counter]['Code'], y.properties)
        y.properties['jobs']=added[counter]
        if (counter<50):
            counter+=1
        
        

newtest.save("../resources/geojsons/datachoropleth.geojson")

In [ ]:
filepath = "https://mw-project-4.s3.us-west-1.amazonaws.com/mw_scraped_indeedjobs.csv"
bigcsv = pd.read_csv(filepath)
alldf = bigcsv.sort_values('Job Category')
alldf

In [ ]:
alldf['Company Name'].value_counts()

In [2]:
govjobs = pd.read_csv('https://mw-project-4.s3.us-west-1.amazonaws.com/mw_govjob_listings_scraped.csv')
govjobs

,Job Title,Company Name,Company Location,Salary,Description,Link,Job Category
0,Data Analyst,Other Agencies and Independent Organizations,"Washington, District of Columbia","Starting at $87,198 (GS 12)",Help\r\n \n...,https://www.usajobs.gov/GetJob/ViewDetails/622...,Data Analyst
1,IT Specialist (DATAMGT) (Data and Reporting An...,Department of the Treasury,No location provided,"Starting at $92,143 (GS 13)",Help\r\n \n...,https://www.usajobs.gov/GetJob/ViewDetails/621...,Data Analyst
2,Program Analyst (Data Science),Department of Veterans Affairs,No location provided,"Starting at $66,829 (GS 12)",Help\r\n \n...,https://www.usajobs.gov/GetJob/ViewDetails/620...,Data Analyst
3,Health System Specialist (Data Analyst),Department of Veterans Affairs,"West Roxbury, Massachusetts","Starting at $59,498 (GS 9-11)",Help\r\n \n...,https://www.usajobs.gov/GetJob/ViewDetails/622...,Data Analyst
4,Interdisciplinary (Data Research Analyst),Other Agencies and Independent Organizations,"Alexandria, Virginia","Starting at $103,690 (AD 03)",Help\r\n \n...,https://www.usajobs.gov/GetJob/ViewDetails/623...,Data Analyst
...,...,...,...,...,...,...,...
11282,Research Economist Research Mathematicia...,Department of the Treasury,"Washington, District of Columbia","Starting at $158,184 (OR 70)",Help\r\n \n...,https://www.usajobs.gov/GetJob/ViewDetails/619...,Statistician
11283,Physical Scientist/Biologist/Mathematical Stat...,Department of the Interior,No location provided,"Starting at $53,433 (GS 9)",Help\r\n \n...,https://www.usajobs.gov/GetJob/ViewDetails/621...,Statistician
11284,Physical Scientist/Biologist/Mathematical Stat...,Department of the Interior,No location provided,"Starting at $77,488 (GS 12)",Help\r\n \n...,https://www.usajobs.gov/GetJob/ViewDetails/621...,Statistician
11285,Health Statistician - PD21E022 - Center for Qu...,Department of Health And Human Services,"Rockville, Maryland","Starting at $103,690 (GS 13)",Help\r\n \n...,https://www.usajobs.gov/GetJob/ViewDetails/620...,Statistician


# WEBSCRAPING GLASSDOOR

In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from webdriver_manager.chrome import ChromeDriverManager

In [7]:
def get_jobs(keyword, num_jobs, verbose):
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    # Initializing the webdriver
    options = webdriver.ChromeOptions()
    # Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    # options.add_argument('headless')
    # Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)
    driver.set_window_size(1120, 1000)
    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + keyword + '&locT=C&locId=1150505&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'    
    driver.get(url)
    global jobs
    jobs = []
    # Let the page load. Change this number based on your internet speed.
    # Or, wait until the webpage is loaded, instead of hardcoding it.
    time.sleep(5)
    # Click on the first job & Test for the "Sign Up" prompt and get rid of it.
    driver.find_element_by_xpath("//*[@id='MainCol']/div[1]/ul/li[1]").click()
    time.sleep(1)
    # Clicking on the Close X button to close the "Sign Up" prompt.
    driver.find_element_by_xpath('//*[@id="JAModal"]/div/div[2]/span').click()  
    time.sleep(5)
    # Clicking on the "Search" button to search for the job postings based on keywords entered.
    driver.find_element_by_xpath('//*[@id="scBar"]/div/button/span').click()
    time.sleep(10)
    while len(jobs) < num_jobs:
        # Going through each job in this page
        job_buttons = driver.find_elements_by_xpath("//*[@id='MainCol']/div[1]/ul/li")
        for job_button in job_buttons:  
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                # When the number of jobs collected has reached the number we set.
                break
            job_button.click()  
            time.sleep(5)
            collected_successfully = False
            while not collected_successfully:
                try:
                    time.sleep(5)
                    company_name = driver.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[1]').text
                    location = driver.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[3]').text
                    job_title = driver.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[2]').text
                    job_description = driver.find_element_by_xpath('//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)
                    collected_successfully = True
            try:
                salary_estimate = driver.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[4]/span').text
            except NoSuchElementException:
                salary_estimate = -1 # You need to set a "not found value. It's important."
            try:
                rating = driver.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[1]/span').text
            except NoSuchElementException:
                rating = -1 # You need to set a "not found value. It's important."
            # Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))
            # Going to the Company tab...
            time.sleep(5)
            try:
                driver.find_element_by_xpath('//div[@data-item="tab" and @data-tab-type="overview"]').click()
                time.sleep(5)
                try:
                    size = driver.find_element_by_xpath('//*[@id="EmpBasicInfo"]/div[1]/div/div[1]/span[2]').text
                except NoSuchElementException:
                    size = -1
                try:
                    founded = driver.find_element_by_xpath('//*[@id="EmpBasicInfo"]/div[1]/div/div[2]/span[2]').text
                except NoSuchElementException:
                    founded = -1
                try:
                    type_of_ownership = driver.find_element_by_xpath('//*[@id="EmpBasicInfo"]/div[1]/div/div[3]/span[2]').text
                except NoSuchElementException:
                    type_of_ownership = -1
                try:
                    industry = driver.find_element_by_xpath('//*[@id="EmpBasicInfo"]/div[1]/div/div[4]/span[2]').text
                except NoSuchElementException:
                    industry = -1
                try:
                    sector = driver.find_element_by_xpath('//*[@id="EmpBasicInfo"]/div[1]/div/div[5]/span[2]').text
                except NoSuchElementException:
                    sector = -1
                try:
                    revenue = driver.find_element_by_xpath('//*[@id="EmpBasicInfo"]/div[1]/div/div[6]/span[2]').text
                except NoSuchElementException:
                    revenue = -1
            except NoSuchElementException:  # Rarely, some job postings do not have the "Company" tab.
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
            if verbose:
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
            jobs.append({
            "Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue
            })
        # Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('//*[@id="FooterPageNav"]/div/ul/li[7]/a/span').click()
            time.sleep(10)
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break
    # This line converts the dictionary object into a pandas DataFrame.
    return pd.DataFrame(jobs)

In [8]:
datawarehousemanager = get_jobs("Data Warehouse Manager", 1000, False)
datawarehousemanager = pd.DataFrame(jobs)
datawarehousemanager.head()



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [C:\Users\Andrew\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


Progress: 0/1000
Progress: 1/1000
Progress: 2/1000
Progress: 3/1000
Progress: 4/1000
Progress: 5/1000
Progress: 6/1000
Progress: 7/1000
Progress: 8/1000
Progress: 9/1000
Progress: 10/1000
Progress: 11/1000
Progress: 12/1000
Progress: 13/1000
Progress: 14/1000
Progress: 15/1000
Progress: 16/1000
Progress: 17/1000
Progress: 18/1000
Progress: 19/1000
Progress: 20/1000
Progress: 21/1000
Progress: 22/1000
Progress: 23/1000
Progress: 24/1000
Progress: 25/1000
Progress: 26/1000
Progress: 27/1000
Progress: 28/1000
Progress: 29/1000
Scraping terminated before reaching target number of jobs. Needed 1000, got 30.


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Technical Project Manager- Data Engineer,$67K - $129K (Glassdoor est.),Overview:\nCharlie’s Produce - Who We Are\n\nC...,3.5,Charlie's Produce\n3.5,"Seattle, WA",1001 to 5000 Employees,1978,Company - Private,Wholesale,Business Services,$500 million to $1 billion (USD)
1,Product Manager - Analytics,$79K - $143K (Glassdoor est.),"For nearly two decades, Zonar Systems has been...",3.2,Zonar Systems\n3.2,"Seattle, WA",201 to 500 Employees,2001,Company - Private,Computer Hardware & Software,Information Technology,$100 to $500 million (USD)
2,Product Manager - Data Engineering,$58K - $125K (Glassdoor est.),"PeopleReady, a TrueBlue company (NYSE: TBI), i...",-1,PeopleReady,"Tacoma, WA",5001 to 10000 Employees,1989,Subsidiary or Business Segment,Staffing & Outsourcing,Business Services,$50 to $100 million (USD)
3,Integration Support Specialist for a market le...,Employer Provided Salary:$80K - $95K,Ticketing system experience\nExcellent trouble...,-1,Zest City,"Seattle, WA",Unknown,Company - Public,Unknown / Non-Applicable,-1,-1,-1
4,"BIE Manager, AWS Data Exchange",-1,"Bachelor’s degree in Business Analytics, Stati...",3.7,"Amazon Web Services, Inc.\n3.7","Seattle, WA",10000+ Employees,1994,Company - Public,Internet,Information Technology,$10+ billion (USD)


In [5]:
datawarehousemanager.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Technical Project Manager- Data Engineer,$67K - $129K (Glassdoor est.),Overview:\nCharlie’s Produce - Who We Are\n\nC...,3.5,Charlie's Produce\n3.5,"Seattle, WA",1001 to 5000 Employees,1978,Company - Private,Wholesale,Business Services,$500 million to $1 billion (USD)
1,Product Manager - Analytics,$79K - $143K (Glassdoor est.),"For nearly two decades, Zonar Systems has been...",3.2,Zonar Systems\n3.2,"Seattle, WA",201 to 500 Employees,2001,Company - Private,Computer Hardware & Software,Information Technology,$100 to $500 million (USD)
2,Product Manager - Data Engineering,$58K - $125K (Glassdoor est.),"PeopleReady, a TrueBlue company (NYSE: TBI), i...",-1,PeopleReady,"Tacoma, WA",5001 to 10000 Employees,1989,Subsidiary or Business Segment,Staffing & Outsourcing,Business Services,$50 to $100 million (USD)
3,Integration Support Specialist for a market le...,Employer Provided Salary:$80K - $95K,Ticketing system experience\nExcellent trouble...,-1,Zest City,"Seattle, WA",Unknown,Company - Public,Unknown / Non-Applicable,-1,-1,-1
4,"BIE Manager, AWS Data Exchange",-1,"Bachelor’s degree in Business Analytics, Stati...",3.7,"Amazon Web Services, Inc.\n3.7","Seattle, WA",10000+ Employees,1994,Company - Public,Internet,Information Technology,$10+ billion (USD)


# SCRAPS

In [ ]:
#setting up splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)

In [ ]:
table_url = "https://www.bls.gov/oes/current/oes_nat.htm"
tablehtml = pd.read_html(table_url)
tableraw = tablehtml[0]
tableraw = tableraw.drop([0, 1041])
tableraw = tableraw.rename(columns = {"Occupation title (click on the occupation title to view its profile)": "Occupation", 'Employment': 'Number_of_Jobs_Listed','Employment per 1,000 jobs': 'Job_Frequency', 'Annual mean wage': 'Salary'})

#table with grouped categories
maincategories = tableraw[tableraw['Occupation code'].str[3].isin(['0'])].reset_index(drop = True)


#cleaned up full table
cleanedtable = tableraw.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]
# cleanedtable
maincategories

In [ ]:
management_occupations = tableraw[tableraw['Occupation code'].str.contains('11-')].reset_index(drop = True)
management_occupations = management_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

business_occupations = tableraw[tableraw['Occupation code'].str.contains('13-')].reset_index(drop = True)
business_occupations = business_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

computer_occupations = tableraw[tableraw['Occupation code'].str.contains('15-')].reset_index(drop = True)
computer_occupations = computer_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

engineering_occupations = tableraw[tableraw['Occupation code'].str.contains('17-')].reset_index(drop = True)
engineering_occupations = engineering_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

science_occupations = tableraw[tableraw['Occupation code'].str.contains('19-')].reset_index(drop = True)
science_occupations = science_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

social_occupations = tableraw[tableraw['Occupation code'].str.contains('21-')].reset_index(drop = True)
social_occupations = social_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

legal_occupations = tableraw[tableraw['Occupation code'].str.contains('23-')].reset_index(drop = True)
legal_occupations = legal_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

education_occupations = tableraw[tableraw['Occupation code'].str.contains('25-')].reset_index(drop = True)
education_occupations = education_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

arts_occupations = tableraw[tableraw['Occupation code'].str.contains('27-')].reset_index(drop = True)
arts_occupations = arts_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

healthcare_occupations = tableraw[tableraw['Occupation code'].str.contains('29-')].reset_index(drop = True)
healthcare_occupations = healthcare_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

healthcaresup_occupations = tableraw[tableraw['Occupation code'].str.contains('31-')].reset_index(drop = True)
healthcaresup_occupations = healthcaresup_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

protective_occupations = tableraw[tableraw['Occupation code'].str.contains('33-')].reset_index(drop = True)
protective_occupations = protective_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

food_occupations = tableraw[tableraw['Occupation code'].str.contains('35-')].reset_index(drop = True)
food_occupations = food_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

building_occupations = tableraw[tableraw['Occupation code'].str.contains('37-')].reset_index(drop = True)
building_occupations = building_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

personalcare_occupations = tableraw[tableraw['Occupation code'].str.contains('39-')].reset_index(drop = True)
personalcare_occupations = personalcare_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

sales_occupations = tableraw[tableraw['Occupation code'].str.contains('41-')].reset_index(drop = True)
sales_occupations = sales_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

office_occupations = tableraw[tableraw['Occupation code'].str.contains('43-')].reset_index(drop = True)
office_occupations = office_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

farming_occupations = tableraw[tableraw['Occupation code'].str.contains('45-')].reset_index(drop = True)
farming_occupations = farming_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

construction_occupations = tableraw[tableraw['Occupation code'].str.contains('47-')].reset_index(drop = True)
construction_occupations = construction_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

maintenance_occupations = tableraw[tableraw['Occupation code'].str.contains('49-')].reset_index(drop = True)
maintenance_occupations = maintenance_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

production_occupations = tableraw[tableraw['Occupation code'].str.contains('51-')].reset_index(drop = True)
production_occupations = production_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

transportation_occupations = tableraw[tableraw['Occupation code'].str.contains('53-')].reset_index(drop = True)
transportation_occupations = transportation_occupations.loc[:, ['Occupation', 'Number_of_Jobs_Listed', 'Job_Frequency', 'Salary']]

# test = transportation_occupations.to_dict('records')
# test

In [ ]:
csvpath = "../resources/choroplethedit.csv"
wagecsv = pd.read_csv(csvpath)
softwarecsv = wagecsv[wagecsv['Occupation text'].str.contains("Software")]
computercsv = wagecsv[wagecsv['Occupation text'].str.contains("Computer")]
newdf = pd.concat([computercsv, softwarecsv])
newdf

In [ ]:
categories = ['Database Admin', 'Big Data Software Engineer', 'Quantitative Analyst', 'IT Systems Analyst', 
             'Data Analyst', 'Business Intelligence Analyst', 'Data Architect', 'Business Systems Analyst', 
             'Machine Learning Engineer', 'Data Engineer', 'Data Scientist', 'Machine Learning Scientist', 
             'Business Intelligence Developer', 'Logistics Analyst', 'Marketing Analyst', 'Data Warehouse Manager', 
             'Statistician', 'Data Storyteller', 'Database Administrator', 'Database Manager', 'Data Modeler']
jobnumbers = [i for i in range(21)]
statenumbers = [i for i in range(50)]
# dfca['Code'] = dfca['Area text'] + dfca['Job Category']


In [ ]:
# nonational = newdf.loc[newdf['Area text'] != 'National']
# df=nonational.sort_values('Occupation text')
# df

In [ ]:
occupation_list = nonational['Occupation text'].unique()
# for x in occupation_list:
    
#     print(x)
len(occupation_list)

In [ ]:
bgroup=bigboi.groupby(['Occupation text','Area text'])
# bgroup["Job characteristic code"].count().reset_index().to_csv("../resources/IgotchuAndrew_sj.csv")
bggroup2 = bgroup["Job characteristic code"].count().reset_index().sort_values('Occupation text')
bggroup2

In [ ]:
states = [dfal, dfak, dfaz, dfar, dfca, dfco, dfct, dfde, dfdc, dffl, dfga, dfhi, dfid, dfil, dfin, dfia, dfks, dfky, dfla, dfme, dfmd, dfma, dfmi, dfmn, dfms, dfmo, dfmt, dfne, dfnv, dfnh, dfnj, dfnm, dfny, dfnc, dfnd, dfoh, dfok, dfor, dfpa, dfri, dfsc, dfsd, dftn, dftx, dfut, dfvt, dfva, dfwa, dfwv, dfwi, dfwy]    
# teststate = dfal.groupby(['Occupation text'])  
# teststate2 = teststate["Job characteristic code"].count().reset_index().sort_values('Occupation text').set_index('Occupation text')
# testdict = teststate2.to_dict()
# testdict
states

In [ ]:
# Alabama = 0
# Alaska = 0
# Arizona = 0
# Arkansas = 0
# California = 0
# Colorado = 0
# Connecticut = 0
# Delaware = 0
# DistrictColumbia = 0
# Florida = 0
# Georgia = 0
# Hawaii = 0
# Idaho = 0
# Illinois = 0
# Indiana = 0
# Iowa =  0
# Kansas = 0
# Kentucky = 0
# Louisiana = 0
# Maine = 0
# Maryland = 0
# Massachusetts = 0
# Michigan = 0
# Minnesota = 0
# Mississippi = 0
# Missouri = 0
# Montana = 0
# Nebraska = 0
# Nevada = 0
# NewHampshire = 0
# NewJersey = 0
# NewMexico = 0
# NewYork = 0
# NorthCarolina = 0
# NorthDakota = 0
# Ohio = 0
# Oklahoma = 0
# Oregon = 0
# Pennsylvania = 0
# RhodeIsland = 0
# SouthCarolina = 0
# SouthDakota = 0
# Tennessee = 0
# Texas = 0
# Utah = 0
# Vermont = 0
# Virginia = 0
# Washington = 0
# WestVirginia = 0
# Wisconsin = 0
# Wyoming = 0